In [2]:
from sklearn_agent import load_database as sklearn_load_database
from pandas_agent import load_database as pandas_load_database
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(),override=True)

True

In [3]:
import os
pandas_database = pandas_load_database(os.environ['OPENAI_API_KEY'])

In [4]:
sklearn_database = sklearn_load_database(os.environ['OPENAI_API_KEY'])

ValueError: Collection sklearn_docs does not exist.